In [1]:
####
## Title: Postal Rollout Merge Data
## Date Created: 9/24/2025
## Description: Load and merge relevant data
## Requirements: 
#####

In [2]:
############################
## Setting Up Environment ##
############################
import pandas as pd 
import numpy as np
import re
from linearmodels import PanelOLS


In [3]:
###############
## Load Data ##
###############
## Load postal rollout data
postal_services_dat = pd.read_csv('../data/raw/postal_service_expansion_US.csv', sep = '\t')

## Load historical patient data from raw patient office
## Notes: Unlinked to anything
# patient_dat = pd.read_csv('../data/historical_patent_masterfile.csv')

## Load historical patient data from Petralia et al. (2023)
## Notes: Located in Harvard Dataverse 
##        Names are a mess      
patient_dat = pd.read_csv('../data/raw/HistPat_Inventors_Assignees.csv')

## State-Abbreviation Cross Walk (maybe I put this into a different file because it's ugly here) 
abbrev_state = {
    "AL": "ALABAMA",
    "AK": "ALASKA",
    "AZ": "ARIZONA",
    "AR": "ARKANSAS",
    "CA": "CALIFORNIA",
    "CO": "COLORADO",
    "CT": "CONNECTICUT",
    "DE": "DELAWARE",
    "FL": "FLORIDA",
    "GA": "GEORGIA",
    "HI": "HAWAII",
    "ID": "IDAHO",
    "IL": "ILLINOIS",
    "IN": "INDIANA",
    "IA": "IOWA",
    "KS": "KANSAS",
    "KY": "KENTUCKY",
    "LA": "LOUISIANA",
    "ME": "MAINE",
    "MD": "MARYLAND",
    "MA": "MASSACHUSETTS",
    "MI": "MICHIGAN",
    "MN": "MINNESOTA",
    "MS": "MISSISSIPPI",
    "MO": "MISSOURI",
    "MT": "MONTANA",
    "NE": "NEBRASKA",
    "NV": "NEVADA",
    "NH": "NEW HAMPSHIRE",
    "NJ": "NEW JERSEY",
    "NM": "NEW MEXICO",
    "NY": "NEW YORK",
    "NC": "NORTH CAROLINA",
    "ND": "NORTH DAKOTA",
    "OH": "OHIO",
    "OK": "OKLAHOMA",
    "OR": "OREGON",
    "PA": "PENNSYLVANIA",
    "RI": "RHODE ISLAND",
    "SC": "SOUTH CAROLINA",
    "SD": "SOUTH DAKOTA",
    "TN": "TENNESSEE",
    "TX": "TEXAS",
    "UT": "UTAH",
    "VT": "VERMONT",
    "VA": "VIRGINIA",
    "WA": "WASHINGTON",
    "WV": "WEST VIRGINIA",
    "WI": "WISCONSIN",
    "WY": "WYOMING",
    "DC": "DISTRICT OF COLUMBIA"
}



In [4]:
###########################
## Cleaning Patient Data ## 
###########################
## Cleaning city and state names 
patient_dat['City'] = patient_dat['City'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z]','',x).lower().replace(" ",""))
patient_dat['State_Name'] = patient_dat['State'].astype(str).map(abbrev_state)  
patient_dat['State_Name'] = patient_dat['State_Name'].astype(str).apply(lambda x: x.lower().replace(" ",""))

## Keeping Patients from the US
patient_dat = patient_dat[~(patient_dat['City'].isin(["","*", "**"]))] 
patient_dat = patient_dat[~(patient_dat['State_Name']=='nan')].reset_index(drop=True)

## Pulling all unique combinations of city and state
unique_patient_city_states = patient_dat[['City', 'State_Name']].drop_duplicates() 

## Collapsing patient data by the city, state, and year (just tally the totals to get the full number)
agg_patient_dat = patient_dat.groupby(['City', 'State_Name','Year']).agg(
    N_patients = ('PN','nunique')
).reset_index()



In [5]:
###########################
## Cleaning Rollout Data ##
###########################
## Turn the town into lowercase 
postal_services_dat['Town'] = postal_services_dat['Town'].astype(str).apply(lambda x: x.lower().replace(" ",""))

## Pull the year from the town and state of the rolled out data 
postal_services_dat['Date']  = pd.to_datetime(postal_services_dat['Date'], format = '%m/%d/%Y')
postal_services_dat['Year'] = postal_services_dat['Date'].dt.year 
postal_services_dat.rename(columns={'Date': 'Rollout_Date', 'Year': 'Rollout_Year'}, inplace=True)

## Checking the number of rural cities/towns that NEVER have a patient from ~1839 to 1973   
check_merge             = postal_services_dat.merge(unique_patient_city_states, left_on = ['Town','State'], right_on = ['City','State_Name'], how = 'inner')
rolled_cities_w_patient = check_merge.shape[0]  
total_cities_rolled     = postal_services_dat.shape[0]
print('Rolled cities with a patient from 1839 to 1973: ', rolled_cities_w_patient)
print('Rolled cities: ', total_cities_rolled)


Rolled cities with a patient from 1839 to 1973:  10718
Rolled cities:  12419


In [6]:
###################################################
## Construct Panel Data for Rollout Towns/Cities ##
###################################################
## Create panel dataset of cities and years from the start of the period to the end
min_year = min(patient_dat['Year'])
max_year = max(patient_dat['Year'])

panel_dat = postal_services_dat.copy()

## Expanding the number of observations by the years thata our panel covers 
panel_dat['N_expand_row'] = max_year - min_year + 2   
panel_dat = panel_dat.loc[panel_dat.index.repeat(panel_dat['N_expand_row'])].reset_index(drop=True)

## Adding in the counted years
panel_dat['year_index'] = panel_dat.groupby(['Town', 'State']).cumcount() 
panel_dat['Year'] = panel_dat['year_index'] + min_year 

## Merging the total number of patients in
panel_dat = panel_dat.merge(agg_patient_dat, left_on =['Town', 'State', 'Year'], right_on =['City', 'State_Name','Year'], how = 'left')
panel_dat = panel_dat.drop(columns = ['City', 'State_Name'])

## Filling in zeros when no patients are filed
panel_dat.loc[panel_dat['N_patients'].isna(), 'N_patients'] = 0
panel_dat['has_patient'] = (panel_dat['N_patients'] >0)*1

## Checking the count
checker = panel_dat[panel_dat['N_patients'] > 0]
print(checker[['Town','State']].drop_duplicates().shape[0]) # We lose a few obs, not sure why... Need to double check


10634


In [7]:
############################################
## Creating Panel Data of Non-Rolled Data ##
############################################
## Setting up the inital panel
min_year = min(patient_dat['Year'])
max_year = max(patient_dat['Year'])
nroll_panel_dat = unique_patient_city_states.copy()
nroll_panel_dat = nroll_panel_dat.rename(columns = {'City':'Town', 'State_Name':'State'})

nroll_panel_dat['N_expand_row'] = max_year-min_year + 1
nroll_panel_dat = nroll_panel_dat.loc[nroll_panel_dat.index.repeat(nroll_panel_dat['N_expand_row'])].reset_index(drop=True)

## Creating year variable
nroll_panel_dat['year_index'] = nroll_panel_dat.groupby(['Town','State']).cumcount()
nroll_panel_dat['Year'] = nroll_panel_dat['year_index'] + min_year

## Merging on the total patients in 
nroll_panel_dat = nroll_panel_dat.merge(agg_patient_dat, 
                                        left_on  = ['Town' , 'State', 'Year'], 
                                        right_on = ['City', 'State_Name', 'Year'],
                                        how = 'left' )
nroll_panel_dat = nroll_panel_dat.drop(columns = ['City', 'State_Name'])
nroll_panel_dat.loc[nroll_panel_dat['N_patients'].isna(),'N_patients'] = 0
nroll_panel_dat['has_patient'] = (nroll_panel_dat['N_patients']>0)*1

## Dropping the data that have a change in the postal service access 
## Aka we seek to keep a constant sample of towns/cities that have never seen a change in postal service access 
## (at least since the time frame that we care about, which let's say is like 1870s onwards)
roll_town_state = postal_services_dat[['Town', 'State']].drop_duplicates()
roll_town_state['check'] = 1
nroll_panel_dat = nroll_panel_dat.merge(roll_town_state, 
                                        on = ['Town', 'State'], 
                                        how = 'left')
nroll_panel_dat = nroll_panel_dat.loc[nroll_panel_dat['check'] !=1].reset_index(drop=True)

## Cleaning Some
nroll_panel_dat.drop(columns = ['check'])


,Town,State,N_expand_row,year_index,Year,N_patients,has_patient
0,lincoln,maine,143,0,1836,1.0,1
1,lincoln,maine,143,1,1837,4.0,1
2,lincoln,maine,143,2,1838,1.0,1
3,lincoln,maine,143,3,1839,2.0,1
4,lincoln,maine,143,4,1840,0.0,0
...,...,...,...,...,...,...,...
5733866,isleofwight,virginia,143,138,1974,0.0,0
5733867,isleofwight,virginia,143,139,1975,0.0,0
5733868,isleofwight,virginia,143,140,1976,0.0,0
5733869,isleofwight,virginia,143,141,1977,0.0,0


In [8]:
nroll_panel_dat.columns

Index(['Town', 'State', 'N_expand_row', 'year_index', 'Year', 'N_patients',
       'has_patient', 'check'],
      dtype='object')

In [9]:
panel_dat.columns

Index(['Town', 'Rollout_Date', 'State', 'Rollout_Year', 'N_expand_row',
       'year_index', 'Year', 'N_patients', 'has_patient'],
      dtype='object')

In [10]:
#################
## Saving Data ##
#################
## Exporting Rolled Panel data
panel_dat.to_csv('../data/clean/postal_rollout_panel.csv', index = False)

## Exporting  Non-Rolled Panel Data
nroll_panel_dat.to_csv('../data/clean/non_postal_rollout_panel.csv', index = False)